In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import os

# Set dataset paths
train_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\train"
valid_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\valid"
test_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\test"

# Image augmentation & preprocessing
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode="binary"
)

val_generator = datagen.flow_from_directory(
    valid_dir, target_size=(224, 224), batch_size=32, class_mode="binary"
)

test_generator = datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode="binary"
)

# Load MobileNetV2 (pre-trained model)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(1, activation="sigmoid")(x)  # Binary classification (E-Waste / Non-E-Waste)

# Build model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save trained model
model.save("E-Waste_Non-Ewaste_classifier.h5")

print("✅ Model trained and saved as 'E-Waste_Non-Ewaste_classifier.h5'")


Found 1800 images belonging to 2 classes.
Found 899 images belonging to 2 classes.
Found 899 images belonging to 2 classes.



Epoch 1/10


57/57 [==============================] - 103s 2s/step - loss: 1.4498 - accuracy: 0.8911 - val_loss: 0.0634 - val_accuracy: 0.9822
Epoch 2/10
57/57 [==============================] - 70s 1s/step - loss: 0.0391 - accuracy: 0.9867 - val_loss: 0.0324 - val_accuracy: 0.9911
Epoch 3/10
57/57 [==============================] - 70s 1s/step - loss: 0.0211 - accuracy: 0.9928 - val_loss: 0.0224 - val_accuracy: 0.9922
Epoch 4/10
57/57 [==============================] - 66s 1s/step - loss: 0.0153 - accuracy: 0.9967 - val_loss: 0.0253 - val_accuracy: 0.9944
Epoch 5/10
57/57 [==============================] - 68s 1s/step - loss: 0.0061 - accuracy: 0.9989 - val_loss: 0.0257 - val_accuracy: 0.9933
Epoch 6/10
57/57 [==============================] - 67s 1s/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.0213 - val_accuracy: 0.9956
Epoch 7/10
57/57 [============

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Model trained and saved as 'E-Waste_Non-Ewaste_classifier.h5'


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

# Set dataset paths
train_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\train"
valid_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\valid"
test_dir = r"C:\Users\lenovo\OneDrive\Desktop\E-WASTE ANALYSIS AND PREDICTING RECYCLING METHOD\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\class\test"

# ✅ Data Augmentation (Prevents overfitting)
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,       # Rotate images up to 30 degrees
    width_shift_range=0.2,   # Shift width by 20%
    height_shift_range=0.2,  # Shift height by 20%
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Zoom in by 20%
    horizontal_flip=True,    # Flip horizontally
    fill_mode="nearest"
)

# Load Data
train_generator = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode="binary")
val_generator = datagen.flow_from_directory(valid_dir, target_size=(224, 224), batch_size=32, class_mode="binary")
test_generator = datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode="binary")

# ✅ Load Pre-trained MobileNetV2 Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# ✅ Unfreeze last 5 layers for better feature extraction
for layer in base_model.layers[:-5]:  
    layer.trainable = False  

# ✅ Improved Classification Layers
x = GlobalAveragePooling2D()(base_model.output)  # Replaces Flatten
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)  # Increased dropout to reduce overfitting
x = Dense(1, activation="sigmoid")(x)  # Binary classification (E-Waste / Non-E-Waste)

# Build Model
model = Model(inputs=base_model.input, outputs=x)

# ✅ Compile Model with Better Learning Rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss="binary_crossentropy", 
              metrics=["accuracy"])

# ✅ Train Model for 15 Epochs
model.fit(train_generator, validation_data=val_generator, epochs=15)

# Save Model
model.save("E-Waste_Non-Ewaste_classifier.h5")

print("✅ Model trained and saved as 'E-Waste_Non-Ewaste_classifier.h5'")


Found 1800 images belonging to 2 classes.
Found 899 images belonging to 2 classes.
Found 899 images belonging to 2 classes.
Epoch 1/15
57/57 [==============================] - 141s 2s/step - loss: 0.2853 - accuracy: 0.8722 - val_loss: 0.2791 - val_accuracy: 0.8977
Epoch 2/15
57/57 [==============================] - 99s 2s/step - loss: 0.1190 - accuracy: 0.9656 - val_loss: 0.2691 - val_accuracy: 0.8954
Epoch 3/15
57/57 [==============================] - 127s 2s/step - loss: 0.0748 - accuracy: 0.9778 - val_loss: 0.2014 - val_accuracy: 0.9143
Epoch 4/15
57/57 [==============================] - 120s 2s/step - loss: 0.0528 - accuracy: 0.9850 - val_loss: 0.2039 - val_accuracy: 0.9177
Epoch 5/15
57/57 [==============================] - 106s 2s/step - loss: 0.0381 - accuracy: 0.9883 - val_loss: 0.2322 - val_accuracy: 0.9188
Epoch 6/15
57/57 [==============================] - 96s 2s/step - loss: 0.0314 - accuracy: 0.9906 - val_loss: 0.1715 - val_accuracy: 0.9399
Epoch 7/15
57/57 [==============